In [ ]:
%matplotlib notebook
%pylab

## Timing and accuracy results for DG, CG and WPA
<hr style="border-width:4px; border-color:coral"></hr>

In [ ]:
import pandas 

In [ ]:
idx = pandas.IndexSlice

# Walltime     : time TFinal (global time)
# Prep         : one time step up time
# Integration  : 
# time steps   : Number of time steps to time tfinal
# dt           : Time step size
# CFL          : CFL number
# Errors       : (1-norm, 2-norm and inf-norm) errors

ex_list = ['advection','SWE']

cols = ['walltime', 'Prep', 'Integ', 'time steps', 'dt', 'cfl','1-norm','2-norm','inf-norm']

order = [1,2,4]
elements = [8,16,32,64,128,256,512, 1024, 2048]
iterables = [ex_list, ['DG','CG','WPA'],['exact','inexact','w/lim','wout/lim'], order, elements]

index = pandas.MultiIndex.from_product(iterables,names=['example','method','integration','order','elements'])
df = pandas.DataFrame(index=index,columns=cols).sort_index()

df.loc[idx['SWE','CG',['exact','inexact']]]

## Read in binary WPA data
<hr style="border-width:2px; border-color:black"></hr>

Read in data created during simulation.  The data is output in binary using a Numpy dynamic type `dtype`.  The data has to then be read in using the same format.  The file structure is : 

    Row 1  : nlen               Number of N values in convergence study

The following block then repeats four times for order=(1,2), limiter=(0,1)

Block :

      dtype([('order',int),('limiter',int)])   
      
      # Followed by N rows of 
      dtype([('N',int),('wall','d'), ('setup','d'),('integ','d'),('M',int),('dt','d'),('cfl','d'),
                  ('1-norm','d'),('2-norm','d'),('inf-norm','d')])
                  


In [ ]:
output_file = 'WPA/wpa_adv_data.dat'

# Open file as a binary file
fout = open(output_file,"rb")  
nlen = fromfile(fout,dtype=int,count=1)

# Create a data type for storing results;  use 'd' for double precision entries
dt_data = dtype([('N',int),('wall','d'), ('setup','d'),('integ','d'),('M',int),('dt','d'),('cfl','d'),
                  ('1-norm','d'),('2-norm','d'),('inf-norm','d')])  

# Create a nested data type
dt_info = dtype([('order',int),('limiter',int)])
dt_block = dtype([('info',dt_info),('data',(dt_data,nlen))])

# Read in four sets of data (orders 1,2 and limiters 0,1)
wpa_data = fromfile(fout,dtype=dt_block, count=4)
fout.close()

cols = ['walltime','Prep','Integ','time steps','dt', 'cfl', '1-norm','2-norm','inf-norm']

# Store data in DataFrame
for k in range(4):     # order=(1,2) x lim=(0,1)
    order = wpa_data[k]['info']['order']
    lim = wpa_data[k]['info']['limiter']
    if lim == 0:        
        limidx = 'wout/lim'
    else:
        limidx = 'w/lim'
    
    for d in wpa_data[k]['data']:
        N = d[0]
        a = [d[i] for i in range(1,len(d))]
        df.loc[idx['advection','WPA',limidx,order,N],cols] = a


# Format the data when displaying the DataFrame

fstr = {'walltime' : "{:.4e}".format, 
        'Prep' : "{:.2e}".format,
        "dt" : "{:.4e}", 
        'Integ' : "{:.4e}".format,
        'cfl' : "{:.2f}".format, 
        '1-norm' : '{:.4e}'.format,
        '2-norm' : '{:.4e}'.format,
        'inf-norm' : '{:.4e}'.format}
        
df_wpa = df.loc[idx['advection','WPA',['w/lim','wout/lim'],1:2],:]
df_wpa.style.format(fstr)

## Read in binary CG/DG data
<hr style="border-width:2px; border-color:black"></hr>

Read in data created during simulation.  The data is output in binary using a Numpy dynamic type `dtype`.  The data has to then be read in using the same format.  The file structure is : 

    Row 1  : nlen               Number of N values in convergence study

The following block then repeats four times for order=(1,2), limiter=(0,1)

Block :

      dtype([('order',int),('limiter',int)])   
      
      # Followed by N rows of 
      dtype([('N',int),('wall','d'), ('setup','d'),('integ','d'),('M',int),('dt','d'),('cfl','d'),
                  ('1-norm','d'),('2-norm','d'),('inf-norm','d')])
                  


In [ ]:
# Read DG/CG data here

output_file = 'CGDG/CG_adv_data.dat'

# Open file as a binary file
fout = open(output_file,"rb")  
nlen = fromfile(fout,dtype=int,count=1)

# Create a data type for storing results;  use 'd' for double precision entries
dt_data = dtype([('N',int),('wall','d'), ('setup','d'),('integ','d'),('M',int),('dt','d'),('cfl','d'),
                  ('1-norm','d'),('2-norm','d'),('inf-norm','d')])  

# Create a nested data type
dt_info = dtype([('order',int)])
dt_block = dtype([('info',dt_info),('data',(dt_data,nlen))])

# Read in four sets of data (orders 1,2 and limiters 0,1)
cgdgdata = fromfile(fout,dtype=dt_block, count=4)
fout.close()

cols = ['walltime','Prep','Integ','time steps','dt', 'cfl', '1-norm','2-norm','inf-norm']

# Store data in DataFrame
for k in range(4):     # order=(1,6) x exact=(0,1)
    order = cgdg_data[k]['info']['order']
    lim = cgdg_data[k]['info']
    if lim == 0:        
        limidx = 'inexact'
    else:
        limidx = 'exact'
    
    for d in wpa_data[k]['data']:
        N = d[0]
        a = [d[i] for i in range(1,len(d))]
        df.loc[idx['advection','CG',limidx,order,N],cols] = a


# Format the data when displaying the DataFrame

fstr = {'walltime' : "{:.4e}".format, 
        'Prep' : "{:.2e}".format,
        "dt" : "{:.4e}", 
        'Integ' : "{:.4e}".format,
        'cfl' : "{:.2f}".format, 
        '1-norm' : '{:.4e}'.format,
        '2-norm' : '{:.4e}'.format,
        'inf-norm' : '{:.4e}'.format}
        
df_cgdg = df.loc[idx['advection','CG',['exact','inexact'],1:2],:]
df_cgdg.style.format(fstr)

## Add extra columns
<hr style="border-width:2px; border-color:black"></hr>

In [ ]:
# Add a DOF column

integration = df.index.levels[2].values
orders = df.index.levels[3].values
elements = df.index.levels[4].values

for o in orders:
    # Is this right?  
    for I in integration:
        df.loc[idx['advection','DG',I,o,:],'DOF'] = (o + 1)*array(elements)
        df.loc[idx['advection','CG',I,o,:],'DOF'] = o*array(elements) + 1
    
# One degree of freedom per element
df.loc[idx['advection','WPA','w/lim',1,:],'DOF'] = elements
df.loc[idx['advection','WPA','w/lim',2,:],'DOF'] = elements

# display data frame
df.loc[idx['advection','DG','exact',:,:],:]

## Using a MultiIndex table

<hr style="border-width:4px; border-color:coral"/>

In [ ]:
# Extract all of the data for one example
df.loc['advection']

In [ ]:
df.loc[('advection','WPA',('w/lim','wout/lim')),:]

In [ ]:
df.loc[('advection','DG',('exact')),['walltime','Prep','1-norm','cfl']]


In [ ]:
cols = ['walltime','Prep','time steps','1-norm'] 
df.loc[idx['advection',:,('exact','wout/lim')],cols].unstack(3)

In [ ]:
df.loc[idx['advection',('DG','CG'),'exact',2],:].\
       style.background_gradient(subset=['walltime'],cmap='YlOrBr')

## Plotting : CG vs. DG
<hr style="border-width:4px; border-color:coral"></hr>

In [ ]:
# set up nice tick marks for log data
def set_xticks(P):
    p0 = log2(P[0])
    p1 = log2(P[-1])
    xlim([2**(p0-1), 2**(p1+1)])
    
    Pstr = (['{:d}'.format(int(p)) for p in P])
    xticks(P,Pstr)

In [ ]:
figure(1)
clf()

# Retrieve element values - level 3 in the Multiindex
Nv = df.index.levels[4].values

# Retrieve time values 
time_dg = df.loc[idx['advection','WPA','w/lim',2,:],'walltime'].values
time_cg = df.loc[idx['advection','WPA','wout/lim',2,:],'walltime'].values

loglog(Nv,time_cg,'r.-',markersize=10,label='DG')
loglog(Nv,time_dg,'b.-',markersize=10,label='CG')

set_xticks(Nv)

title('Timing comparison between CG and CG')
xlabel('# of elements')
ylabel('Walltime (s)')

legend()